### TODO:
1. ellenőrizni megyéket: lett évről-évre eltérés? --> ha nincs nagy eltérés, hozzuk egy oszlopra
2. adatokat aggregálni megye illetve gazdálkodási forma és ágazat kategória szerint
3. feature_importance check
4. cross-validation
5. number of trees to use: plot the training: num trees on x and error on y axis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [14]:
from sklearn.model_selection import train_test_split

target_column = 'target_reg'

train_df = pd.read_csv('./dataset/train.csv')
evaluation_df = pd.read_csv('./dataset/test.csv')

# Clean data, reduce features

In [15]:
train_df.dropna(inplace=True)

In [16]:
MEGYE_COLUMNS = ['MEGYE_2014', 'MEGYE_2015', 'MEGYE_2016']
SZEKHELY_COLUMNS = ['SZEKHELY_IR_SZAM_2014', 'SZEKHELY_IR_SZAM_2015', 'SZEKHELY_IR_SZAM_2016']

GAZD_FORMA_FO_KAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_fo_kategoria', 'GAZDALKODASI_FORMA_2015_fo_kategoria', 'GAZDALKODASI_FORMA_2016_fo_kategoria']
GAZD_FORMA_1_ALKAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_1_alkategoria', 'GAZDALKODASI_FORMA_2015_1_alkategoria', 'GAZDALKODASI_FORMA_2016_1_alkategoria']
GAZD_FORMA_2_ALKAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_2_alkategoria', 'GAZDALKODASI_FORMA_2015_2_alkategoria', 'GAZDALKODASI_FORMA_2016_2_alkategoria']

KKV_BESOR_FO_KAT_COLUMNS = ['KKV_BESOROLAS_2014_fo_kategoria', 'KKV_BESOROLAS_2015_fo_kategoria', 'KKV_BESOROLAS_2016_fo_kategoria']
KKV_BESOR_1_ALKAT_COLUMNS = ['KKV_BESOROLAS_2014_1_alkategoria', 'KKV_BESOROLAS_2015_1_alkategoria', 'KKV_BESOROLAS_2016_1_alkategoria']
KKV_BESOR_2_ALKAT_COLUMNS = ['KKV_BESOROLAS_2014_2_alkategoria', 'KKV_BESOROLAS_2015_2_alkategoria', 'KKV_BESOROLAS_2016_2_alkategoria']

AGAZAT_FO_KAT_COLUMNS = ['agazat_2014_fo_kategoria', 'agazat_2015_fo_kategoria', 'agazat_2016_fo_kategoria'] 
AGAZAT_1_ALKAT_COLUMNS = ['agazat_2014_1_alkategoria', 'agazat_2015_1_alkategoria', 'agazat_2016_1_alkategoria']
AGAZAT_2_ALKAT_COLUMNS = ['agazat_2014_2_alkategoria', 'agazat_2015_2_alkategoria', 'agazat_2016_2_alkategoria']
AGAZAT_3_ALKAT_COLUMNS = ['agazat_2014_3_alkategoria', 'agazat_2015_3_alkategoria', 'agazat_2016_3_alkategoria']

COLUMN_GROUPS = [
    MEGYE_COLUMNS, 
    SZEKHELY_COLUMNS,
    GAZD_FORMA_FO_KAT_COLUMNS,
    GAZD_FORMA_1_ALKAT_COLUMNS,
    GAZD_FORMA_2_ALKAT_COLUMNS,
    KKV_BESOR_FO_KAT_COLUMNS,
    KKV_BESOR_1_ALKAT_COLUMNS,
    KKV_BESOR_2_ALKAT_COLUMNS,
    AGAZAT_FO_KAT_COLUMNS,
    AGAZAT_1_ALKAT_COLUMNS,
    AGAZAT_2_ALKAT_COLUMNS,
    AGAZAT_3_ALKAT_COLUMNS,
]

In [17]:
from itertools import combinations


def check_if_difference_in_columns(df, columns):
    for pair in combinations(columns, 2):
        column_a, column_b = pair[0], pair[1]
        differing_number_of_rows = df.loc[df[column_a] != df[column_b]].shape[0]
        if differing_number_of_rows > 0:
            print(
                f'different number of rows: {differing_number_of_rows}',
                f'found in columns: {column_a}, {column_b}',
                sep=' '
            )
        else:
            print(f'no difference in columns: {column_a}, {column_b}')

    return False

def check_column_groups_for_difference(df, column_groups):
    for group in column_groups:
        check_if_difference_in_columns(df, group)




In [18]:
evaluation_df.loc[evaluation_df['KKV_BESOROLAS_2015_fo_kategoria'] != evaluation_df['KKV_BESOROLAS_2016_fo_kategoria']].shape[0]

16

In [19]:
check_column_groups_for_difference(df=train_df, column_groups=COLUMN_GROUPS)

no difference in columns: MEGYE_2014, MEGYE_2015
no difference in columns: MEGYE_2014, MEGYE_2016
no difference in columns: MEGYE_2015, MEGYE_2016
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2015
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2016
no difference in columns: SZEKHELY_IR_SZAM_2015, SZEKHELY_IR_SZAM_2016
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2015_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2015_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2015_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2015_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GA

In [20]:
check_column_groups_for_difference(df=evaluation_df, column_groups=COLUMN_GROUPS)

no difference in columns: MEGYE_2014, MEGYE_2015
no difference in columns: MEGYE_2014, MEGYE_2016
no difference in columns: MEGYE_2015, MEGYE_2016
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2015
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2016
no difference in columns: SZEKHELY_IR_SZAM_2015, SZEKHELY_IR_SZAM_2016
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2015_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2015_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2015_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2015_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GA

since some of the column groups have no difference among them, we can keep one column per group and drop the rest

In [21]:
train_df_reduced_features = train_df.copy()

column_groups_to_reduce = [
    MEGYE_COLUMNS,
    SZEKHELY_COLUMNS,
    GAZD_FORMA_FO_KAT_COLUMNS,
    GAZD_FORMA_1_ALKAT_COLUMNS,
    GAZD_FORMA_2_ALKAT_COLUMNS
]

for group in column_groups_to_reduce:
    train_df_reduced_features.drop(columns=group[1:], axis='columns', inplace=True)

pd.set_option('display.max_columns', None)
train_df_reduced_features.head()

,id,target_reg,KATEGORIKUS_VALTOZO_2014,KATEGORIKUS_VALTOZO_2015,KATEGORIKUS_VALTOZO_2016,MEGYE_2014,SZEKHELY_IR_SZAM_2014,aktelh_2014,aktelh_2015,aktelh_2016,aktsajt_2014,aktsajt_2015,aktsajt_2016,alakul_ev,arbevert_2014,arbevert_2015,arbevert_2016,arbevexp_2014,arbevexp_2015,arbevexp_2016,befpue_2014,befpue_2015,befpue_2016,begyeb_2014,begyeb_2015,begyeb_2016,celtart_2014,celtart_2015,celtart_2016,eredadel_2014,eredadel_2015,eredadel_2016,eredadoz_2014,eredadoz_2015,eredadoz_2016,eredpenz_2014,eredpenz_2015,eredpenz_2016,eredtar_2014,eredtar_2015,eredtar_2016,ereduzem_2014,ereduzem_2015,ereduzem_2016,ertpapir_2014,ertpapir_2015,ertpapir_2016,hoskot01_2014,hoskot01_2015,hoskot01_2016,hoskot_2014,hoskot_2015,hoskot_2016,immat_2014,immat_2015,immat_2016,jetok_2014,jetok_2015,jetok_2016,kecs_2014,kecs_2015,kecs_2016,keszl_2014,keszl_2015,keszl_2016,kovet_2014,kovet_2015,kovet_2016,letszam_2014,letszam_2015,letszam_2016,merlfoo_2014,merlfoo_2015,merlfoo_2016,penzeszk_2014,penzeszk_2015,penzeszk_2016,ranyag_2014,ranyag_2015,ranyag_2016,regyeb_2014,regyeb_2015,regyeb_2016,rovkot01_2014,rovkot01_2015,rovkot01_2016,rovkot_2014,rovkot_2015,rovkot_2016,rszem_2014,rszem_2015,rszem_2016,sajattok_2014,sajattok_2015,sajattok_2016,targyie_2014,targyie_2015,targyie_2016,GAZDALKODASI_FORMA_2014_fo_kategoria,GAZDALKODASI_FORMA_2014_1_alkategoria,GAZDALKODASI_FORMA_2014_2_alkategoria,KKV_BESOROLAS_2014_fo_kategoria,KKV_BESOROLAS_2014_1_alkategoria,KKV_BESOROLAS_2014_2_alkategoria,KKV_BESOROLAS_2015_fo_kategoria,KKV_BESOROLAS_2015_1_alkategoria,KKV_BESOROLAS_2015_2_alkategoria,KKV_BESOROLAS_2016_fo_kategoria,KKV_BESOROLAS_2016_1_alkategoria,KKV_BESOROLAS_2016_2_alkategoria,agazat_2014_fo_kategoria,agazat_2014_1_alkategoria,agazat_2014_2_alkategoria,agazat_2014_3_alkategoria,agazat_2015_fo_kategoria,agazat_2015_1_alkategoria,agazat_2015_2_alkategoria,agazat_2015_3_alkategoria,agazat_2016_fo_kategoria,agazat_2016_1_alkategoria,agazat_2016_2_alkategoria,agazat_2016_3_alkategoria
0,1,1907229,0,0,0,Győr-Moson-Sopron,zipcode_1,457724,586098,643830,535126,849583,1234907,1976.0,13983828,12699378,11432950,1382989,1700751,1529118,1855172,1849289,1839898,15456185,15568368,19590889,1490863,2806727,3665800,939106,773314,703532,706406,433306,500046,-145397,35709,-83352,37983621,55695087,89228704,-2661421,-2520379,786884,0,0,0,952490,521178,2885261,48333543,45717629,45942043,132219,103981,84122,7520242,7520242,7520242,5547891,5849322,5686293,1395646,1567766,1798279,9012948,7550810,6199791,1845,1849,1845,121166809,145682514,170549985,1181661,3878508,3592889,14700476,12925752,13101480,2144006,2135752,1412180,9752906,3854551,5805061,19908755,27579737,17326862,10244187,10726882,11271908,13450028,13883334,14386576,107131440,130146063,156391175,P,P,A9,7-R,A,A,7-R,A,A,7-R,A,A,A9,9,P,A,A9,9,P,A,A9,9,P,A
1,2,0,0,0,0,Pest,zipcode_521,49,720,421,0,0,0,2005.0,41717,51525,60916,0,0,0,0,0,0,104,9346,0,0,0,0,7723,3361,56,6950,3024,-66,22,-110,-428,0,0,0,7701,3471,484,0,0,0,0,5026,3693,0,5026,3693,0,0,338,5497,5497,5497,2536,1891,3463,0,0,0,5200,6087,5327,7,6,6,32023,44129,40447,23968,27858,26844,19031,35572,45404,1059,9354,1059,1724,4614,3496,1724,5778,3496,11494,10583,10506,30300,33324,33258,2806,9464,7518,P,P,x,P,P,x,P,P,x,P,P,x,w-5,w-5,7-R,7-R,w-5,w-5,7-R,7-R,w-5,w-5,7-R,7-R
2,3,0,0,0,0,Fejér,zipcode_30,80,538,680,0,0,0,2003.0,181193,146301,156244,0,0,0,0,0,0,7538,0,0,0,0,0,119017,67182,76190,107115,60323,68571,1116,386,106,0,0,0,117806,67126,76085,0,0,0,0,0,0,0,0,0,0,0,0,3301,3301,3301,4112,6862,5454,0,0,0,3713,5178,5927,12,14,14,305680,314230,520744,139298,152725,361724,22130,25566,27486,3652,2978,3020,0,0,0,120245,123494,261437,41030,43769,44200,185435,190736,259307,162589,155789,152413,P,x,w-5,P,7-R,A,P,7-R,A,P,7-R,A,w-5,9,P,A,w-5,9,P,A,w-5,9,P,A
3,4,0,0,0,0,Hajdú-Bihar,zipcode_241,203,265,232,0,0,0,2005.0,76856,83338,83230,0,0,0,0,0,0,403,2219,1136,0,0,0,616,598,239,589,545,187,-1209,-991,-323,8501,8059,7734,1608,1152,562,0,0,0,4646,2414,183,11233,12030,13280,0,0,0,3

### target_reg min miért negatív? létezik negatív export? + plottal meg lehetne vizsgálni

In [22]:
print(train_df.describe())

                 id    target_reg  KATEGORIKUS_VALTOZO_2014  \
count  31745.000000  3.174500e+04              31745.000000   
mean   16388.415404  2.469644e+05                  0.081997   
std     9573.589176  2.466065e+06                  0.274365   
min        1.000000 -1.650300e+04                  0.000000   
25%     8139.000000  0.000000e+00                  0.000000   
50%    16196.000000  0.000000e+00                  0.000000   
75%    24672.000000  7.800000e+01                  0.000000   
max    33180.000000  1.047418e+08                  1.000000   

       KATEGORIKUS_VALTOZO_2015  KATEGORIKUS_VALTOZO_2016   aktelh_2014  \
count              31745.000000              31745.000000  3.174500e+04   
mean                   0.079666                  0.078343  1.644770e+04   
std                    0.270780                  0.268715  2.977193e+05   
min                    0.000000                  0.000000  0.000000e+00   
25%                    0.000000                  0.000000

In [23]:
from sklearn.preprocessing import OneHotEncoder
from utils import columns_to_onehotencode

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())
train_df.reset_index(inplace=True)
encoder_df.reset_index(inplace=True)
train_df = pd.concat([train_df, encoder_df], axis='columns')
print(train_df.shape)
print(train_df.isna().sum().sum())

(31745, 6704)
0


# Train

In [24]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=75, random_state=42, n_jobs= 2)
rf.fit(x_train,y_train)
predictions = rf.predict(x_test)

# Inspecting feature importance

In [26]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(rf, x_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(train_df.feature_names[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

KeyboardInterrupt: 

# Evaluate

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(predictions, y_test)
mae = mean_absolute_error(predictions, y_test)
print(f'mean squared error: {mse}')
print(f'mean absolute error: {mae}')